In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim

### Run all models

In [2]:
sim.run(["build_networks"]) #initialize network accessibility engine
sim.run(["scheduled_development_events", "neighborhood_vars", #scheduled events and accessibility variables
         "rsh_simulate", "nrh_simulate", "nrh_simulate2",   #price models
         "jobs_transition", "elcm_simulate", "households_transition", "hlcm_luz_simulate", #demand/location models
         "price_vars", "feasibility", "residential_developer", "non_residential_developer", #supply/proforma models
         "model_integration_indicators", #indicators for pasef/pecas
         "buildings_to_uc", #export buildings to urban canvas
         ], years=range(2013, 2016))

Running model 'build_networks'
Time to execute model 'build_networks': 13.85s
Total time to execute: 13.85s
Running year 2013
Running model 'scheduled_development_events'
Time to execute model 'scheduled_development_events': 0.75s
Running model 'neighborhood_vars'
Computing accessibility variables
Computing ave_parcel_size
Computing mean_nonres_rent_2000m
Computing jobs_1500m
Computing jobs_800m
Computing jobs_400m
Computing jobs1
Computing jobs2
Computing jobs3
Computing jobs4
Computing jobs5
Computing jobs6
Computing jobs7
Computing jobs8
Computing jobs9
Computing jobs10
Computing jobs11
Computing jobs12
Computing jobs13
Computing jobs14
Computing jobs15
Computing jobs16
Computing jobs17
Computing jobs18
Computing jobs19
Computing jobs20
Computing ave_income
Computing ave_age_of_head_1500m
Computing ave_children_1500m
Computing ave_year_built_1500m
Computing population_400m
Computing job_spaces_3000m
Computing jobs_3000m
Computing households_3000m
Computing residential_units_3000m
Co

c:\Anaconda\lib\site-packages\pandas\util\decorators.py:81: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  warnings.warn(msg, FutureWarning)
c:\Anaconda\lib\site-packages\pandas\util\decorators.py:13: FutureWarning: diff is deprecated. Use difference instead
  FutureWarning)



    and 0 overfull buildings
Time to execute model 'elcm_simulate': 31.31s
Running model 'households_transition'
Time to execute model 'households_transition': 15.59s
Running model 'hlcm_luz_simulate'
There are 100408 total movers for this LCM
HLCM for LUZ 1.0
There are 11314 total available units
    and 444 total choosers
    but there are 0 overfull buildings
    for a total of 4613 temporarily empty units
    in 3286 buildings total in the LUZ
Assigned 444 choosers to new units
Total currently unplaced: 99964
HLCM for LUZ 2.0
There are 12865 total available units
    and 509 total choosers
    but there are 0 overfull buildings
    for a total of 3654 temporarily empty units
    in 3217 buildings total in the LUZ
Assigned 509 choosers to new units
Total currently unplaced: 99455
HLCM for LUZ 3.0
There are 26708 total available units
    and 2114 total choosers
    but there are 0 overfull buildings
    for a total of 10030 temporarily empty units
    in 3416 buildings total in the

c:\Anaconda\lib\site-packages\pandas\core\index.py:670: FutureWarning: slice indexers when using iloc should be integers and not floating point
  "and not floating point",FutureWarning)
c:\Anaconda\lib\site-packages\pandas\computation\expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))



Unplaced jobs after: 4756
Simulated DU: 50904.0
Target DU: 46006
Time to execute model 'residential_developer': 10.51s
Running model 'non_residential_developer'
772,102 feasible buildings before running developer
Job space target for heavy_industrial is -29036.5317706.
Job space target for light_industrial is -155026.14315.
Job space target for retail is 106277.435.
Sum of net units that are profitable: 188,355
Constructed 8090 retail buildings, totaling 109244.65603 new job spaces.
Unplaced households before: 7530
Unplaced households after: 17968
Unplaced jobs before: 4756
Unplaced jobs after: 6018
Job space target for office is 5407.78581.
Sum of net units that are profitable: 27,013
Constructed 411 office buildings, totaling 5474.14400389 new job spaces.
Unplaced households before: 17968
Unplaced households after: 18549
Unplaced jobs before: 6018
Unplaced jobs after: 6071
Time to execute model 'non_residential_developer': 7.61s
Running model 'model_integration_indicators'
Exporting

c:\Anaconda\lib\site-packages\pandas\io\sql.py:1511: FutureWarning: read_frame is deprecated, use read_sql
  warnings.warn("read_frame is deprecated, use read_sql", FutureWarning)


### LUZ Indicators

In [16]:
bsim = sim.get_table('buildings').to_frame(columns = ['luz_id', 'note', 'res_price_per_sqft', 'nonres_rent_per_sqft', 'residential_units', 'non_residential_sqft'])
luz_res_price = bsim[bsim.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = bsim[bsim.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
bsim = bsim[(bsim.note == 'simulated') | bsim.note.str.startswith('Sitespec')]
luz_simdu = bsim.groupby('luz_id').residential_units.sum()
luz_simnr = bsim.groupby('luz_id').non_residential_sqft.sum()
luz_df = pd.DataFrame({'du':luz_simdu, 'nrsf':luz_simnr, 'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
luz_df = luz_df[luz_df.index.values != 0].fillna(0)
luz_df.index = luz_df.index.values.astype('int')
luz_df.index.name = 'luz_id'
from spandex import TableLoader; loader = TableLoader(); luz_df.to_csv(loader.get_path('out/luz_sim_indicators.csv'))

### MSA Indicators

In [12]:
# Summarize results at MSA level
b = sim.get_table('buildings').to_frame(columns = ['msa_id', 'mgra_id', 'residential_units', 'non_residential_sqft', 'note'])
new_du_by_msa = b[b.note  == 'simulated'].groupby('msa_id').residential_units.sum()
new_nrsf_by_msa = b[b.note  == 'simulated'].groupby('msa_id').non_residential_sqft.sum()
proportion_du_by_msa = new_du_by_msa / new_du_by_msa.sum()
proportion_nrsf_by_msa = new_nrsf_by_msa / new_nrsf_by_msa.sum()
print proportion_du_by_msa
print proportion_nrsf_by_msa 

msa_id
0         0.389515
1         0.399239
2         0.102267
3         0.007191
4         0.043335
5         0.058452
Name: residential_units, dtype: float64
msa_id
0         0.181940
1         0.399495
2         0.187885
3         0.006874
4         0.159979
5         0.063828
Name: non_residential_sqft, dtype: float64


### Predict prices and summarize/export at the LUZ level

In [ ]:
%%capture
## Accessibility engine
sim.run(["build_networks", "neighborhood_vars"])

# Predict prices
sim.run(["rsh_simulate", "nrh_simulate", "nrh_simulate2"])

##Export LUZ mean prices to csv for visual checking
buildings = sim.get_table('buildings').to_frame()
luz_res_price = buildings[buildings.residential_units > 0].groupby('luz_id').res_price_per_sqft.mean()
luz_nonres_price = buildings[buildings.non_residential_sqft > 0].groupby('luz_id').nonres_rent_per_sqft.mean()
df = pd.DataFrame({'res_price':luz_res_price, 'nonres_price':luz_nonres_price})
# df.fillna(0).to_csv(loader.get_path('out/sim/sd_prices.csv'))

# Summarize prices using network queries
sim.run(['price_vars'])

### Proforma

In [ ]:
# from urbansim.developer import sqftproforma
# # Step 1:  feasibility
sim.run(['feasibility'])
sim.run(["residential_developer"])
sim.run(["non_residential_developer"])